# MongoDB

### Setting things up

Remember to restart the kernel after running this code cell if you didn't already have pymongo installed (you only need to do this once)

In [7]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import pprint

In [22]:
client = MongoClient('localhost', 27017)

In [23]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [24]:
db = client.hotel

In [61]:
guests = pd.read_csv('Project 2 Launch  - Guests.csv')
payments = pd.read_csv('Project 2 Launch  - Payments.csv')
reservations = pd.read_csv('Project 2 Launch  - Reservations.csv')
rooms = pd.read_csv('Project 2 Launch  - Rooms.csv')

In [50]:
reservations.columns

Index(['Reservation_ID', 'Room_Number', 'Guest_ID', 'Checkin_Date',
       'Checkout_Date'],
      dtype='object')

In [62]:
# guests_collection = db.guests
# guests_data = guests.to_dict(orient='records')
# guests_collection.insert_many(guests_data)

# payments_collection = db.payments
# payments_data = payments.to_dict(orient='records')
# payments_collection.insert_many(payments_data)

reservations_collection = db.reservations
reservations_data = reservations.to_dict(orient='records')
reservations_collection.insert_many(reservations_data)

rooms_collection = db.rooms
rooms_data = rooms.to_dict(orient='records')
rooms_collection.insert_many(rooms_data)

InsertManyResult([ObjectId('665746973a8b63f9eac6bf66'), ObjectId('665746973a8b63f9eac6bf67'), ObjectId('665746973a8b63f9eac6bf68'), ObjectId('665746973a8b63f9eac6bf69'), ObjectId('665746973a8b63f9eac6bf6a'), ObjectId('665746973a8b63f9eac6bf6b'), ObjectId('665746973a8b63f9eac6bf6c'), ObjectId('665746973a8b63f9eac6bf6d'), ObjectId('665746973a8b63f9eac6bf6e'), ObjectId('665746973a8b63f9eac6bf6f'), ObjectId('665746973a8b63f9eac6bf70'), ObjectId('665746973a8b63f9eac6bf71'), ObjectId('665746973a8b63f9eac6bf72'), ObjectId('665746973a8b63f9eac6bf73'), ObjectId('665746973a8b63f9eac6bf74'), ObjectId('665746973a8b63f9eac6bf75'), ObjectId('665746973a8b63f9eac6bf76'), ObjectId('665746973a8b63f9eac6bf77'), ObjectId('665746973a8b63f9eac6bf78'), ObjectId('665746973a8b63f9eac6bf79'), ObjectId('665746973a8b63f9eac6bf7a'), ObjectId('665746973a8b63f9eac6bf7b'), ObjectId('665746973a8b63f9eac6bf7c'), ObjectId('665746973a8b63f9eac6bf7d'), ObjectId('665746973a8b63f9eac6bf7e'), ObjectId('665746973a8b63f9eac6bf

MongoDB is made up of, in a simplified way, massive JSON objects

### Querying data

Let's explore some example data that might be similar to the type you are creating in your projects

We can easily convert flat data to dictionary format with `to_dict()`

We can then insert it into our MongoDB

### Which rooms have the highest occupancy rates? (Rooms<->Reservation)

In [63]:
pipeline = [
    {
        '$lookup': {
            'from': 'reservations',
            'localField': 'ID',
            'foreignField': 'Room_Number',
            'as': 'reservations'
        }
    },
    {
        '$unwind': '$reservations'
    },
    {
        '$addFields': {
            'checkinDate': {
                '$dateFromString': {
                    'dateString': '$reservations.Checkin_Date',
                    'format': '%m/%d/%Y'
                }
            },
            'checkoutDate': {
                '$dateFromString': {
                    'dateString': '$reservations.Checkout_Date',
                    'format': '%m/%d/%Y'
                }
            }
        }
    },
    {
        '$group': {
            '_id': {
                'roomnum': '$ID',
                'type': '$Type',
                'capacity': '$Capacity',
                'price': '$Price'
            },
            'totalBookedDays': {
                '$sum': { '$subtract': ['$checkoutDate', '$checkinDate'] }
            },
            'totalReservations': {
                '$sum': 1
            }
        }
    },
    {
        '$lookup': {
            'from': 'reservations',
            'pipeline': [
                {
                    '$project': {
                        'checkinDate': {
                            '$dateFromString': {
                                'dateString': '$Checkin_Date',
                                'format': '%m/%d/%Y'
                            }
                        },
                        'checkoutDate': {
                            '$dateFromString': {
                                'dateString': '$Checkout_Date',
                                'format': '%m/%d/%Y'
                            }
                        }
                    }
                }
            ],
            'as': 'allReservations'
        }
    },
    {
        '$unwind': '$allReservations'
    },
    {
        '$group': {
            '_id': None,
            'minDate': { '$min': '$allReservations.checkinDate' },
            'maxDate': { '$max': '$allReservations.checkoutDate' }
        }
    },
    {
        '$addFields': {
            'totalDays': {
                '$subtract': ['$maxDate', '$minDate']
            }
        }
    },
    {
        '$lookup': {
            'from': 'rooms',
            'let': { 'minDate': '$minDate', 'maxDate': '$maxDate', 'totalDays': '$totalDays' },
            'pipeline': [
                {
                    '$lookup': {
                        'from': 'reservations',
                        'localField': 'ID',
                        'foreignField': 'Room_Number',
                        'as': 'reservations'
                    }
                },
                {
                    '$unwind': '$reservations'
                },
                {
                    '$addFields': {
                        'checkinDate': {
                            '$dateFromString': {
                                'dateString': '$reservations.Checkin_Date',
                                'format': '%m/%d/%Y'
                            }
                        },
                        'checkoutDate': {
                            '$dateFromString': {
                                'dateString': '$reservations.Checkout_Date',
                                'format': '%m/%d/%Y'
                            }
                        }
                    }
                },
                {
                    '$group': {
                        '_id': {
                            'roomnum': '$ID',
                            'type': '$Type',
                            'capacity': '$Capacity',
                            'price': '$Price'
                        },
                        'totalBookedDays': {
                            '$sum': { '$subtract': ['$checkoutDate', '$checkinDate'] }
                        },
                        'totalReservations': {
                            '$sum': 1
                        }
                    }
                },
                {
                    '$addFields': {
                        'occupancyRate': {
                            '$multiply': [
                                {
                                    '$divide': ['$totalBookedDays', '$$totalDays']
                                },
                                100
                            ]
                        }
                    }
                },
                {
                    '$sort': { 'occupancyRate': -1 }
                }
            ],
            'as': 'roomsOccupancy'
        }
    },
    {
        '$unwind': '$roomsOccupancy'
    },
    {
        '$replaceRoot': { 'newRoot': '$roomsOccupancy' }
    }
]

In [64]:


result = list(rooms_collection.aggregate(pipeline))

for r in result:
    print(r)

{'_id': {'roomnum': 8, 'type': 'Double', 'capacity': 2, 'price': 150}, 'totalBookedDays': 1468800000, 'totalReservations': 2, 'occupancyRate': 94.44444444444444}
{'_id': {'roomnum': 20, 'type': 'Double', 'capacity': 2, 'price': 150}, 'totalBookedDays': 1382400000, 'totalReservations': 2, 'occupancyRate': 88.88888888888889}
{'_id': {'roomnum': 3, 'type': 'Suite', 'capacity': 4, 'price': 300}, 'totalBookedDays': 1296000000, 'totalReservations': 2, 'occupancyRate': 83.33333333333334}
{'_id': {'roomnum': 10, 'type': 'Single', 'capacity': 1, 'price': 100}, 'totalBookedDays': 1123200000, 'totalReservations': 2, 'occupancyRate': 72.22222222222221}
{'_id': {'roomnum': 6, 'type': 'Suite', 'capacity': 4, 'price': 300}, 'totalBookedDays': 1123200000, 'totalReservations': 2, 'occupancyRate': 72.22222222222221}
{'_id': {'roomnum': 19, 'type': 'Single', 'capacity': 1, 'price': 100}, 'totalBookedDays': 1036800000, 'totalReservations': 2, 'occupancyRate': 66.66666666666666}
{'_id': {'roomnum': 4, 'typ

In [70]:
import time
start_time = time.time()
result = list(rooms_collection.aggregate(pipeline))
end_time = time.time()
execution_time = end_time - start_time
print(f"MongoDB query execution time: {execution_time} seconds")

MongoDB query execution time: 0.006505489349365234 seconds


We can use prettyprint to make things a bit more human readable

In [8]:
query = {'Type': 'Single'}
result = rooms_collection.find(query)

In [9]:
for e in result:
    pprint.pprint(e)

{'Capacity': 1,
 'ID': 1,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb5d')}
{'Capacity': 1,
 'ID': 4,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb60')}
{'Capacity': 1,
 'ID': 7,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb63')}
{'Capacity': 1,
 'ID': 10,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb66')}
{'Capacity': 1,
 'ID': 13,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb69')}
{'Capacity': 1,
 'ID': 16,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb6c')}
{'Capacity': 1,
 'ID': 19,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('66562feb55685df22d8bcb6f')}
{'Capacity': 1,
 'ID': 1,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('665738483a8b63f9eac6bee3')}
{'Capacity': 1,
 'ID': 4,
 'Price': 100,
 'Type': 'Single',
 '_id': ObjectId('665738483a8b63f9eac6bee6')}
{'Capacity': 1,
 'ID': 7,
 'Price': 100,
 